# Chapter 7: Data Cleaning and Preparation

In [4]:
import pandas as pd

In [5]:
import numpy as np

## 7.1 Handling Missing Data

Missing data occurs commonly in many data analysis applications. One of the goals of pandas is to make working with missing data as painless as possible. For example, all of the descriptive statistics on pandas objects exclude missing data by default.

The way that missing data is represented in pandas objects is somewhat imperfect, but it is functional for a lot of users. For numeric data, pandas uses the floating-point value NaN (Not a Number) to represent missing data. We call this a sentinel value that can be easily detected:

In [6]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avacado'])

In [7]:
string_data

0     aardvark
1    artichoke
2          NaN
3      avacado
dtype: object

In [8]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In pandas, we’ve adopted a convention used in the R programming language by refer‐ ring to missing data as NA, which stands for not available. In statistics applications, NA data may either be data that does not exist or that exists but was not observed (through problems with data collection, for example). When cleaning up data for analysis, it is often important to do analysis on the missing data itself to identify data collection problems or potential biases in the data caused by missing data.

The built-in Python None value is also treated as NA in object arrays:

In [9]:
string_data[0] = None

In [10]:
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

## Filtering Out Missing Data

There are a few ways to filter out missing data. While you always have the option to do it by hand using pandas.isnull and boolean indexing, the dropna can be helpful. On a Series, it returns the Series with only the non-null data and index values:

In [11]:
from numpy import nan as NA

In [12]:
data = pd.Series([1, NA, 3.5, NA, 7])

In [13]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

<b>This is equivalent to:

In [14]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

<b>With DataFrame objects, things are a bit more complex. You may want to drop rows or columns that are all NA or only those containing any NAs. dropna by default drops any row containing a missing value:

In [15]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA], [NA, NA, NA], [NA, 6.5, 3.]])

In [16]:
cleaned = data.dropna()

In [17]:
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [18]:
cleaned

,0,1,2
0,1.0,6.5,3.0


<b>Passing how='all' will only drop rows that are all NA

In [19]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


<b>To drop columns in the same way, pass axis = 1

In [20]:
data[4] = NA

In [21]:
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [22]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


<b>A related way to filter out DataFrame rows tends to concern time series data. Suppose you want to keep only rows containing a certain number of observations. You can indicate this with the thresh argument:

In [23]:
df = pd.DataFrame(np.random.randn(7, 3))

In [24]:
df.iloc[:4, 1] = NA

In [25]:
df.iloc[:2, 2] = NA

In [26]:
df

,0,1,2
0,-2.518367,NaN,NaN
1,-0.018171,NaN,NaN
2,1.301002,NaN,0.668309
3,-1.801792,NaN,0.318047
4,-0.880365,-1.956347,0.193552
5,-0.433336,-1.090629,0.655778
6,0.029162,1.545327,-0.949774


In [27]:
df.dropna()

,0,1,2
4,-0.880365,-1.956347,0.193552
5,-0.433336,-1.090629,0.655778
6,0.029162,1.545327,-0.949774


In [28]:
df.dropna(thresh=2)

,0,1,2
2,1.301002,NaN,0.668309
3,-1.801792,NaN,0.318047
4,-0.880365,-1.956347,0.193552
5,-0.433336,-1.090629,0.655778
6,0.029162,1.545327,-0.949774


## Filling In Missing Data

Rather than filtering out missing data (and potentially discarding other data along with it), you may want to fill in the “holes” in any number of ways. For most pur‐ poses, the fillna method is the workhorse function to use. Calling fillna with a constant replaces missing values with that value:

In [29]:
df.fillna(0)

,0,1,2
0,-2.518367,0.000000,0.000000
1,-0.018171,0.000000,0.000000
2,1.301002,0.000000,0.668309
3,-1.801792,0.000000,0.318047
4,-0.880365,-1.956347,0.193552
5,-0.433336,-1.090629,0.655778
6,0.029162,1.545327,-0.949774


<b>Calling fillna with a dict, you can use a different fill value for each column:

In [30]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-2.518367,0.500000,0.000000
1,-0.018171,0.500000,0.000000
2,1.301002,0.500000,0.668309
3,-1.801792,0.500000,0.318047
4,-0.880365,-1.956347,0.193552
5,-0.433336,-1.090629,0.655778
6,0.029162,1.545327,-0.949774


<b>fillna returns a new object, but you can modify the existing object in-place:

In [31]:
_ = df.fillna(0, inplace=True)

In [32]:
df

,0,1,2
0,-2.518367,0.000000,0.000000
1,-0.018171,0.000000,0.000000
2,1.301002,0.000000,0.668309
3,-1.801792,0.000000,0.318047
4,-0.880365,-1.956347,0.193552
5,-0.433336,-1.090629,0.655778
6,0.029162,1.545327,-0.949774


<b>The same interpolation methods available for reindexing can be used with fillna:

In [33]:
df = pd.DataFrame(np.random.randn(6, 3))

In [34]:
df.iloc[2:, 1] = NA

In [35]:
df.iloc[4:, 2] = NA

In [36]:
df

,0,1,2
0,0.786955,-0.696099,-0.569753
1,-1.145801,0.097043,0.340875
2,0.298451,NaN,0.673030
3,0.246507,NaN,-0.779681
4,-0.491224,NaN,NaN
5,-0.447217,NaN,NaN


In [37]:
df.fillna(method='ffill')

,0,1,2
0,0.786955,-0.696099,-0.569753
1,-1.145801,0.097043,0.340875
2,0.298451,0.097043,0.673030
3,0.246507,0.097043,-0.779681
4,-0.491224,0.097043,-0.779681
5,-0.447217,0.097043,-0.779681


In [38]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,0.786955,-0.696099,-0.569753
1,-1.145801,0.097043,0.340875
2,0.298451,0.097043,0.673030
3,0.246507,0.097043,-0.779681
4,-0.491224,NaN,-0.779681
5,-0.447217,NaN,-0.779681


<b>With fillna you can do lots of other things with a little creativity. For example, you might pass the mean or median value of a Series:

In [39]:
data = pd.Series([1., NA, 3.5, NA, 7])

In [40]:
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

## 7.2 Data Transformation

### Removing Duplicates

<b>Duplicate rows may be found in a DataFrame for any number of reasons. Here is an example:

In [41]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'], 'k2': [1, 1, 2, 3, 3, 4, 4]})

In [42]:
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


<b>The DataFrame method duplicated returns a boolean Series indicating whether each row is a duplicate (has been observed in a previous row) or not:

In [43]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

<b>Relatedly, drop_duplicates returns a DataFrame where the duplicated array is False:

In [44]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


<b>Both of these methods by default consider all of the columns; alternatively, you can specify any subset of them to detect duplicates. Suppose we had an additional column of values and wanted to filter duplicates only based on the 'k1' column:

In [45]:
data['v1'] = range(7)

In [46]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


<b>duplicated and drop_duplicates by default keep the first observed value combina‐ tion. Passing keep='last' will return the last one:

In [47]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


## Transforming Data Using a Function or Mapping

For many datasets, you may wish to perform some transformation based on the values in an array, Series, or column in a DataFrame. Consider the following hypothetical data collected about various kinds of meat:

In [48]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami', 'corned beef', 'Bacon', 'pastrami', 'honey ham', 'nova lox'], 'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

In [49]:
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


<b>Suppose you wanted to add a column indicating the type of animal that each food came from. Let’s write down a mapping of each distinct meat type to the kind of animal:

In [50]:
 meat_to_animal = {
      'bacon': 'pig',
      'pulled pork': 'pig',
      'pastrami': 'cow',
      'corned beef': 'cow',
      'honey ham': 'pig',
      'nova lox': 'salmon'
}

<b>The map method on a Series accepts a function or dict-like object containing a map‐ ping, but here we have a small problem in that some of the meats are capitalized and others are not. Thus, we need to convert each value to lowercase using the str.lower Series method:

In [51]:
lowercased = data['food'].str.lower()

In [52]:
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [53]:
data['animal'] = lowercased.map(meat_to_animal)

In [54]:
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


<b>We could also have passed a function that does all the work:

In [55]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

<b>Using map is a convenient way to perform element-wise transformations and other data cleaning–related operations.

## Replacing Values

Filling in missing data with the fillna method is a special case of more general value replacement. As you’ve already seen, map can be used to modify a subset of values in an object but replace provides a simpler and more flexible way to do so. Let’s con‐ sider this Series:

In [56]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])

In [57]:
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

<b>The -999 values might be sentinel values for missing data. To replace these with NA values that pandas understands, we can use replace, producing a new Series (unless you pass inplace=True):

In [58]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

<b>If you want to replace multiple values at once, you instead pass a list and then the
substitute value:

In [59]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

<b>To use a different replacement for each value, pass a list of substitutes:

In [60]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

<b>The argument passed can also be a dict:

In [61]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

## Renaming Axis Indexes

Like values in a Series, axis labels can be similarly transformed by a function or map‐ ping of some form to produce new, differently labeled objects. You can also modify the axes in-place without creating a new data structure. Here’s a simple example:

In [62]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)), index=['Ohio', 'Colorado', 'New York'], columns=['one', 'two', 'three', 'four'])

<b>Like a Series, the axis indexes have a map method:

In [63]:
transform = lambda x: x[:4].upper()

In [64]:
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

<b>You can assign to index, modifying the DataFrame in place:

In [65]:
data.index = data.index.map(transform)

In [66]:
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


<b>If you want to create a transformed version of a dataset without modifying the origi‐ nal, a useful method is rename:

In [67]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


<b>Notably, rename can be used in conjunction with a dict-like object providing new val‐ ues for a subset of the axis labels:

In [68]:
data.rename(index={'OHIO':'INDIANA'}, columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


<b>rename saves you from the chore of copying the DataFrame manually and assigning to its index and columns attributes. Should you wish to modify a dataset in-place, pass inplace=True:

In [69]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)

In [70]:
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


## Discretization and Binning

Continuous data is often discretized or otherwise separated into “bins” for analysis. Suppose you have data about a group of people in a study, and you want to group them into discrete age buckets:

In [71]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

<b>Let’s divide these into bins of 18 to 25, 26 to 35, 36 to 60, and finally 61 and older. To
do so, you have to use cut, a function in pandas:

In [72]:
bins = [18, 25, 35, 60, 100]

In [73]:
cats = pd.cut(ages, bins)

In [74]:
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

<b>The object pandas returns is a special Categorical object. The output you see describes the bins computed by pandas.cut. You can treat it like an array of strings indicating the bin name; internally it contains a categories array specifying the dis‐ tinct category names along with a labeling for the ages data in the codes attribute:

In [75]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [76]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [77]:
pd.value_counts(cats)

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

<b>Note that pd.value_counts(cats) are the bin counts for the result of pandas.cut.</b>

Consistent with mathematical notation for intervals, a parenthesis means that the side is open, while the square bracket means it is closed (inclusive). You can change which side is closed by passing right=False:

In [78]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64, left]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

<b>You can also pass your own bin names by passing a list or array to the labels option:

In [79]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']

In [80]:
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

<b>
If you pass an integer number of bins to cut instead of explicit bin edges, it will com‐ pute equal-length bins based on the minimum and maximum values in the data. Consider the case of some uniformly distributed data chopped into fourths:

In [81]:
data = np.random.rand(20)

In [82]:
pd.cut(data, 4, precision=2)

[(0.76, 1.0], (0.28, 0.52], (0.036, 0.28], (0.76, 1.0], (0.76, 1.0], ..., (0.52, 0.76], (0.036, 0.28], (0.036, 0.28], (0.76, 1.0], (0.76, 1.0]]
Length: 20
Categories (4, interval[float64, right]): [(0.036, 0.28] < (0.28, 0.52] < (0.52, 0.76] < (0.76, 1.0]]

<b>The precision=2 option limits the decimal precision to two digits.</b>

A closely related function, qcut, bins the data based on sample quantiles. Depending on the distribution of the data, using cut will not usually result in each bin having the same number of data points. Since qcut uses sample quantiles instead, by definition you will obtain roughly equal-size bins:

In [83]:
data = np.random.randn(1000) # normally distributed

In [84]:
cats = pd.qcut(data, 4)

In [85]:
cats

[(-0.629, 0.0155], (-3.145, -0.629], (-0.629, 0.0155], (0.685, 3.258], (0.0155, 0.685], ..., (-0.629, 0.0155], (-3.145, -0.629], (-0.629, 0.0155], (-0.629, 0.0155], (0.685, 3.258]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.145, -0.629] < (-0.629, 0.0155] < (0.0155, 0.685] < (0.685, 3.258]]

In [86]:
pd.value_counts(cats)

(-3.145, -0.629]    250
(-0.629, 0.0155]    250
(0.0155, 0.685]     250
(0.685, 3.258]      250
dtype: int64

<b>Similiar to cut, you can pass your own quantiles (numbers between 0 & 1, inclusive):

In [87]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(-1.241, 0.0155], (-1.241, 0.0155], (-1.241, 0.0155], (0.0155, 1.326], (0.0155, 1.326], ..., (-1.241, 0.0155], (-1.241, 0.0155], (-1.241, 0.0155], (-1.241, 0.0155], (0.0155, 1.326]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.145, -1.241] < (-1.241, 0.0155] < (0.0155, 1.326] < (1.326, 3.258]]

## Detecting and Filtering Outliers

In [88]:
data = pd.DataFrame(np.random.randn(1000, 4))

In [89]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.012777,0.055545,0.036472,0.075932
std,0.987646,1.015588,1.028606,0.968013
min,-2.881984,-3.488701,-3.370918,-2.602106
25%,-0.666825,-0.658218,-0.649484,-0.580158
50%,-0.046356,0.054063,0.075052,0.065381
75%,0.748800,0.755574,0.689076,0.729652
max,3.051473,3.059467,3.051335,3.672276


<b>Suppose you want to find values in one of the columns exceeding 3 in absolute value:

In [90]:
col = data[2]

In [91]:
col[np.abs(col) > 3]

81    -3.059353
109   -3.125753
718   -3.370918
722    3.051335
Name: 2, dtype: float64

<b>To select all rows having a value exceeding 3 or -3, you can use the any method on a boolean DataFrame:

In [92]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
81,-0.563206,-0.205018,-3.059353,-0.727883
109,-0.589718,-0.413261,-3.125753,1.018290
184,0.284991,3.003256,0.062613,1.223468
405,0.544818,-3.488701,0.354127,-0.902920
559,3.051473,-0.211309,0.059208,0.094047
655,0.457193,3.059467,-1.331714,-0.139782
682,-0.196531,0.213457,0.978491,3.672276
718,-0.779050,0.146060,-3.370918,0.666341
722,-0.251165,-1.293951,3.051335,0.604952
983,1.431539,-3.001705,-1.848071,0.423515


<b>Values can be set based on these criteria. Here is code to cap values outside the interval -3 to 3:

In [93]:
data[np.abs(data) > 3] = np.sign(data) * 3

In [94]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.012726,0.055973,0.036977,0.075260
std,0.987489,1.013808,1.026735,0.965744
min,-2.881984,-3.000000,-3.000000,-2.602106
25%,-0.666825,-0.658218,-0.649484,-0.580158
50%,-0.046356,0.054063,0.075052,0.065381
75%,0.748800,0.755574,0.689076,0.729652
max,3.000000,3.000000,3.000000,3.000000


<b>The statement np.sign(data) produces 1 and -1 values based on whether the values in data are positive or negative:

In [95]:
np.sign(data).head()

,0,1,2,3
0,1.0,-1.0,-1.0,1.0
1,1.0,1.0,-1.0,-1.0
2,1.0,1.0,-1.0,1.0
3,-1.0,1.0,1.0,1.0
4,-1.0,-1.0,-1.0,-1.0


## Permutation and Random Sampling

Permuting (randomly reordering) a Series or the rows in a DataFrame is easy to do using the numpy.random.permutation function. Calling permutation with the length of the axis you want to permute produces an array of integers indicating the new ordering:

In [96]:
df = pd.DataFrame(np.arange(5*4).reshape((5,4)))

In [97]:
sampler = np.random.permutation(5)

In [98]:
sampler

array([4, 1, 0, 2, 3])

<b>That array can then be used in iloc-based indexing or the euivalent take function:

In [99]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [100]:
df.take(sampler)

,0,1,2,3
4,16,17,18,19
1,4,5,6,7
0,0,1,2,3
2,8,9,10,11
3,12,13,14,15


<b>To select a random subset without replacement, you can use the sample method on Series and DataFrame:

In [101]:
df.sample(n=3)

,0,1,2,3
4,16,17,18,19
3,12,13,14,15
2,8,9,10,11


<b>To generate a sample with replacement (to allow repeat choices), pass replace=True to sample:

In [102]:
choices = pd.Series([5, 7, -1, 6, 4])

In [103]:
draws = choices.sample(n=10, replace=True)

In [104]:
draws

1    7
0    5
3    6
1    7
1    7
3    6
0    5
1    7
2   -1
3    6
dtype: int64

## Computing Idicator/Dummy Variables

Another type of transformation for statistical modeling or machine learning applica‐ tions is converting a categorical variable into a “dummy” or “indicator” matrix. If a column in a DataFrame has k distinct values, you would derive a matrix or Data‐ Frame with k columns containing all 1s and 0s. pandas has a get_dummies function for doing this, though devising one yourself is not difficult. Let’s return to an earlier example DataFrame:

In [105]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'], 'data1': range(6)})

In [106]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In some cases, you may want to add a prefix to the columns in the indicator Data‐ Frame, which can then be merged with the other data. get_dummies has a prefix argu‐ ment for doing this:

In [107]:
dummies = pd.get_dummies(df['key'], prefix='key')

In [108]:
df_with_dummy = df[['data1']].join(dummies)

In [109]:
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


If a row in a DataFrame belongs to multiple categories, things are a bit more compli‐ cated. Let’s look at the MovieLens 1M dataset, which is investigated in more detail in Chapter 14:

In [110]:
mnames = ['movie_id', 'title', 'genres']

In [113]:
movies = pd.read_table('datasets/movielens/movies.dat', sep='::', header=None, names=mnames)

In [114]:
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


<b>Adding indicator variables for each genre requires a little bit of wrangling. First, we extract the list of unique genres in the dataset:

In [115]:
all_genres = []

In [116]:
for x in movies.genres:
    all_genres.extend(x.split('|'))

In [117]:
genres = pd.unique(all_genres)

In [118]:
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

<b>One way to construct the indicator DataFrame is to start with a DataFrame of all zeros:

In [119]:
zero_matrix = np.zeros((len(movies), len(genres)))

In [120]:
dummies = pd.DataFrame(zero_matrix, columns=genres)

<b>Now, iterate through each movie and set entries in each row of dummies to 1. To do this, we use the dummies.columns to compute the column indices for each genre:

In [121]:
gen = movies.genres[0]

In [123]:
gen.split('|')

['Animation', "Children's", 'Comedy']

In [124]:
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2])

<b>Then, we can use .iloc to set values based on these indices:

In [125]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

<b>Then, as before, you can combine this with movies:

In [126]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))

In [127]:
movies_windic.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Animation                              1.0
Genre_Children's                             1.0
Genre_Comedy                                 1.0
Genre_Adventure                              0.0
Genre_Fantasy                                0.0
Genre_Romance                                0.0
Genre_Drama                                  0.0
Genre_Action                                 0.0
Genre_Crime                                  0.0
Genre_Thriller                               0.0
Genre_Horror                                 0.0
Genre_Sci-Fi                                 0.0
Genre_Documentary                            0.0
Genre_War                                    0.0
Genre_Musical                                0.0
Genre_Mystery                                0.0
Genre_Film-Noir                              0.0
Genre_Western       

<b>A useful recipe for statistical applications is to combine get_dummies with a discreti‐ zation function like cut:

In [128]:
np.random.seed(12345)

In [129]:
values = np.random.rand(10)

In [130]:
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [131]:
bin = [0, 0.2, 0.4, 0.6, 0.8, 1]

In [133]:
pd.get_dummies(pd.cut(values,bin))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


<b>We set the random seed with numpy.random.seed to make the example deterministic. We will look again at pandas.get_dummies later in the book.

## 7.3 String Manipulation

Python has long been a popular raw data manipulation language in part due to its ease of use for string and text processing. Most text operations are made simple with the string object’s built-in methods. For more complex pattern matching and text manipulations, regular expressions may be needed. pandas adds to the mix by ena‐ bling you to apply string and regular expressions concisely on whole arrays of data, additionally handling the annoyance of missing data.

## String Object Methods

In many string munging and scripting applications, built-in string methods are suffi‐ cient. As an example, a comma-separated string can be broken into pieces with split:

In [134]:
val = 'a,b, guido'

In [135]:
val.split(',')

['a', 'b', ' guido']

<b>split is often combined with strip to trim whitespace (including line breaks):

In [144]:
pieces = [x.strip() for x in val.split(',')]

In [145]:
pieces

['a', 'b', 'guido']

<b>These substrings could be concatenated together with a two-colon delimiter using addition:

In [146]:
first, second, third = pieces

In [147]:
first + '::' + second + '::' + third

'a::b::guido'

<b>But this isn’t a practical generic method. A faster and more Pythonic way is to pass a list or tuple to the join method on the string '::':

In [148]:
'::'.join(pieces)

'a::b::guido'

<b>Other methods are concerned with locating substrings. Using Python’s in keyword is the best way to detect a substring, though index and find can also be used:

In [150]:
'guido' in val

True

In [151]:
val.index(',')

1

In [152]:
val.find(':')

-1

<b>Note the difference between find and index is that index raises an exception if the string isn’t found (versus returning –1):

In [153]:
val.index(':')

ValueError: substring not found

<b>Relatedly, count returns the number of occurrences of a particular substring:

In [154]:
val.count(',')

2

<b>replace will substitute occurrences of one pattern for another. It is commonly used to delete patterns, too, by passing an empty string:

In [155]:
val.replace(',', '::')

'a::b:: guido'

In [156]:
val.replace(',', '')

'ab guido'

## Regular Expressions

In [157]:
import re

In [159]:
text = "foo     bar\t baz  \tqux"

In [160]:
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

<b>When you call re.split('\s+', text), the regular expression is first compiled, and then its split method is called on the passed text. You can compile the regex yourself with re.compile, forming a reusable regex object:

In [161]:
regex = re.compile('\s+')

In [162]:
regex.split(text)

['foo', 'bar', 'baz', 'qux']

<b>If, instead, you wanted to get a list of all patterns matching the regex, you can use the
findall method:

In [163]:
regex.findall(text)

['     ', '\t ', '  \t']

<b>Creating a regex object with re.compile is highly recommended if you intend to apply the same expression to many strings; doing so will save CPU cycles.</b>

match and search are closely related to findall. While findall returns all matches in a string, search returns only the first match. More rigidly, match only matches at the beginning of the string. As a less trivial example, let’s consider a block of text and a regular expression capable of identifying most email addresses:

In [164]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""

In [165]:
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

In [166]:
# re.IGNORECASE makes the regex case-insentive

In [167]:
regex = re.compile(pattern, flags=re.IGNORECASE)

<b>Using findall on the text produces a list of the email addresses:

In [169]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

<b>search returns a special match object for the first email address in the text. For the preceding regex, the match object can only tell us the start and end position of the pattern in the string:

In [170]:
m = regex.search(text)

In [171]:
m

<re.Match object; span=(5, 20), match='dave@google.com'>

In [172]:
text[m.start():m.end()]

'dave@google.com'

<b>regex.match returns None, as it only will match if the pattern occurs at the start of the string:

In [173]:
print(regex.match(text))

None


<b>Relatedly, sub will return a new string with occurrences of the pattern replaced by the a new string:

In [174]:
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



<b>Suppose you wanted to find email addresses and simultaneously segment each address into its three components: username, domain name, and domain suffix. To do this, put parentheses around the parts of the pattern to segment:

In [175]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'

In [176]:
regex = re.compile(pattern, flags=re.IGNORECASE)

<b>A match object produced by this modified regex returns a tuple of the pattern com‐ ponents with its groups method:

In [177]:
m = regex.match('wesm@bright.net')

In [178]:
m.groups()

('wesm', 'bright', 'net')

<b>findall returns a list of tuples when the pattern has groups:

In [179]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

<b>sub also has access to groups in each match using special symbols like \1 and \2. The symbol \1 corresponds to the first matched group, \2 corresponds to the second, and so forth:

In [182]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



## Vectorized String Functions in pandas

In [183]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com', 'Rob': 'rob@gmail.com', 'Wes': np.nan}

In [184]:
data = pd.Series(data)

In [185]:
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [186]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

<b>You can apply string and regular expression methods can be applied (passing a lambda or other function) to each value using data.map, but it will fail on the NA (null) values. To cope with this, Series has array-oriented methods for string opera‐ tions that skip NA values. These are accessed through Series’s str attribute; for exam‐ ple, we could check whether each email address has 'gmail' in it with str.contains:

In [187]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

<b>Regular expressions can be used, too, along with any re options like IGNORECASE:

In [189]:
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [190]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

<b>There are a couple of ways to do vectorized element retrieval. Either use str.get or index into the str attribute:

In [199]:
matches = data.str.match(pattern, flags=re.IGNORECASE)

In [200]:
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

<b>To access elements in the embeeded lists, we can pass an index to either of these functions:

In [204]:
matches.str.get(1)

AttributeError: Can only use .str accessor with string values!

In [202]:
matches.str[0]

AttributeError: Can only use .str accessor with string values!

In [203]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object